# Jupyter Notebook du Projet Velib - MDI230

Fait par CHARPENTIER Antoine, MENIOLLE Alban et MORANTIN Bastien.

Nous commençons par importer toutes les bibliothèques dont nous aurons besoin pour ce projet :

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Données initiales

Tout d'abord, nous commencons par entre les données initiales fournies dans l'énoncé, pour le cas des velibs de Rouen avec 5 stations :

In [2]:
# Données 
N_stations = 5
tau =np.array([[0,3,5,7,7],
     [2,0,2,5,5],
     [4,2,0,3,3],
     [8,6,4,0,2],
     [7,7,5,2,0]])*60
#On passe de minutes en secondes

Lambda = np.array([2.8, 3.7, 5.5, 3.5, 4.6])/3600 
#on passe d'un taux de départ en heures à secondes 

P = np.array([[0, 0.22, 0.32, 0.2, 0.26],
              [0.17, 0, 0.34, 0.21, 0.28],
              [0.19, 0.26, 0, 0.24, 0.31],
              [0.17, 0.22, 0.33, 0, 0.28],
              [0.18, 0.24, 0.35, 0.23, 0]])

N=np.array([20, 15, 17, 13, 18])

Nt=np.array([[0,1,0,0,0],
            [1,0,1,0,0],
            [0,1,0,1,0],
            [0,0,1,0,1],
            [0,0,0,1,0]])

On reprend la même modélisation qu'en classe avec les colonies, on a donc l'espace d'état composé des stations i mais aussi des trajets de i vers j.On a donc: 

    • les lambda i vers tij pour un vélo qui part de la station i pour faire le trajet tij

    • les lambda tij vers j pour un vélo qui arrive à la station j après le trajet tij

In [3]:
#Étant donné qu'il ya beaucoup de tau communs et qu'il y a deux matrices lambda,
#on peut utiliser une fonction plutôt qu'une matrice:

def Lambda_tij_j(i,j):
    return 1/tau[i][j]

def Lambda_i_tij(i,j):
    return Lambda[i]*P[i][j]

Faisons maintenant une première simulation pour les transitions de la chaîne de Markov que nous venons de définir :

In [108]:
state =  np.array([20,0,0,0,0,15,0,0,0,0,17,0,0,0,0,13,0,0,0,0,18,0,0,0,0])

def randChoice(liste,probas):
    n = liste.size
    if(n!=probas.size): print("error in randChoice")
    else:
        rand = np.random.random()
        somme=0
        for i in range(0,n):
            somme=somme+probas[i]
            if(rand<somme): return liste[i]
            
def iteration(state):
    nextState = state.copy()
    dest=np.array([],dtype=int)
    lambdas=np.array([],dtype=float)
    for i in range(0,N_stations):
        dest = np.append(dest,int(i))
        lambdas = np.append(lambdas,0.0)
    for i in range(0,N_stations):
        for j in range(0,N_stations):
            somme=0
            if(state[int(5*i+j)]>0):
                if j==0:
                    for x in range(0,N_stations):
                        #ici, erreur sur la manière dont sont associés les lambdas_i qui correspondent
                        #aux probas d'aller à la gare i mais sont interprétés comme étant lambdas[0]=je reste, etc
                        if(x!=i):
                            lambdas[x]=Lambda_i_tij(i,x)
                            somme = somme + Lambda_i_tij(i,x)
                        else: lambdas[i]=1-Lambda[i]               
                    rand = randChoice(dest,lambdas)
                    #np.random.choice(dest,1,False,lambdas)
                    nextState[int(5*i)]=nextState[int(5*i)]-1
                    index=int(5*i+rand)
                    nextState[index]=nextState[index]+1
                else:
                    rand = np.random.random()
                    if(j<=i):x=j-1
                    else:x=j
                    if(rand>Lambda_tij_j(i,x)):
                        nextState[5*i+j]=nextState[5*i+j]-1
                        nextState[5*j]=nextState[5*j]+1
    return nextState

nextState = iteration(state)
print(nextState)

[20  0  0  0  0 14  1  0  0  0 16  0  1  0  0 12  0  0  1  0 17  0  0  0
  1]


Nous pouvons constater que des vélos sont partis des stations de base en direction d'autres stations, nous avons moins de vélos dans certaines stations et quelques vélo en déplacement. Essayons cette fois de réaliser 150 itérations :

In [115]:
for i in range(0,150):
    state=iteration(state)
print(state)

[ 0  0  0  0  0  2 35  0  0  0  1  0  0  0  0  1  0  0 29  0  2  0  0  0
 13]


In [102]:
state=np.array([[20,15,17,13,18]])
stateHist=state
hist=state
dfStateHist=pd.DataFrame(state)
distr_hist = [[0,0,0]]
for x in range(20):
  state=np.dot(state,P)
  #print(state)
  stateHist= np.append(stateHist,state,axis=0)
  hist = np.append(hist, state)
  dfDistrHist = pd.DataFrame(stateHist)
  #dfDistrHist.plot()
#plt.show()
print(stateHist)

[[20.         15.         17.         13.         18.        ]
 [11.23       16.         22.09       15.37       18.31      ]
 [12.8258     15.9898     20.5142     15.1189     18.5513    ]
 [12.525411   15.933838   21.02298    15.113225   18.404546  ]
 [12.58518519 15.96356576 20.85459179 15.12974896 18.4669083 ]
 [12.57227944 15.95153737 20.91110668 15.12187679 18.44319973]
 [12.57536663 15.95597004 20.89199137 15.12488028 18.45179169]
 [12.57454542 15.95440208 20.89848472 15.12381705 18.44875074]
 [12.57478448 15.95494595 20.89627363 15.12418252 18.44981343]
 [12.57471025 15.95475911 20.89702759 15.1240583  18.44944476]
 [12.57473426 15.954823   20.89677028 15.12410038 18.44957209]
 [12.5747263  15.95480119 20.89685814 15.12408613 18.44952824]
 [12.57472897 15.95480863 20.89682813 15.12409096 18.44954331]
 [12.57472807 15.95480609 20.89683838 15.12408932 18.44953814]
 [12.57472838 15.95480696 20.89683488 15.12408988 18.44953991]
 [12.57472827 15.95480666 20.89683607 15.12408969 18.44